# WEIRDO Solute Transport

WEIRDO solute transport need to be implemented in the DoWaterMovement function, this happens on a hourly or subhourly timestep, when rainfall is very high, 6-min time step.

The process is implemented in a loop through each profile layer through functions such as doInfiltration, doDrainage etc., one way is to also do solute transport by stepping through each layer (may have potential issues). 
Possible solution is to save out deltas (e.g. fluxes) in an array, after each doSomething function was called and implement a doSoluteTransport function.

First link to ISolute and map the solute to WEIRDO.Thickness. 

Work on a mass basis instead of concentration basis


## Variables to create

1. Mapped solutes 
    - create in mapped soil water properties and in MapVariable function
    - LeachedNH4, Urea and NO3 are already declared in WEIRDO
    - SoilWater.WaterBalance solutes only include NO3 and Urea

In [ ]:
# LINK TO SOLUTE
        [Link(ByName = true)]
        private ISolute NO3 = null;

        [Link(ByName = true)]
        private ISolute NH4 = null;

        [Link(ByName = true)]
        private ISolute Urea = null;

        [Link(ByName = true, IsOptional = true)]
        ISolute cl = null;
        
# MAPPED VARIABLES
        /// <summary>Mapped from parameter set onto Layer structure</summary>
        public double[] MappedNH4 { get; set; }

        /// <summary>Mapped from parameter set onto Layer structure</summary>
        public double[] MappedUrea { get; set; }

        /// <summary>Mapped from parameter set onto Layer structure</summary>
        public double[] MappedNO3 { get; set; }
        
# ALLOW OUTPUT TO INSPECT
        ///<summary> NO3 in solute in each layer </summary>
        public double[] No3Values { get; set; }
        ///<summary> Urea in solute in each layer </summary>
        public double[] UreaValues { get; set; }
        
# PROPERTIES enable targetThickness to map

        [NonSerialized]
        private double[] targetThickness;

In [ ]:
# on simulation commencing, declare and initialise the variables
            No3Values = new double[ProfileLayers];
            UreaValues = new double[ProfileLayers];
            SoluteFlowEfficiency = new double[ProfileLayers];
            SoluteFluxEfficiency = new double[ProfileLayers];
            
# need to allow console input of fluxEfficiency? 
            if (SoluteFluxEfficiency == null)
                SoluteFluxEfficiency = Enumerable.Repeat<double>(1, ProfileLayers).ToArray();
            if (SoluteFlowEfficiency == null)
                SoluteFlowEfficiency = Enumerable.Repeat<double>(1, ProfileLayers).ToArray();

In [ ]:
        /// <summary>
        /// Called at the start of each daily timestep
        /// </summary>
        /// <param name="sender"></param>
        /// <param name="e"></param>
        [EventSubscribe("DoDailyInitialisation")]
        private void OnDoDailyInitialisation(object sender, EventArgs e)
        {
            Irrigation = 0;
            IrrigationDuration = 0;
            Rainfall = 0;
            Drainage = 0;
            Infiltration = 0;
            pond_evap = 0;
            Es = 0;
            WaterExtraction = 0;
            double CropCover = 0;
            if(Plant != null)
                if (Plant.Leaf != null)
                    CropCover = Plant.Leaf.CoverTotal;
            TotalCover = Math.Min(1, SurfaceOM.Cover + CropCover);
            double SoilRadn = Met.Radn * (1-TotalCover);
            double WindRun = Met.Wind * 86400 / 1000 * (1 - TotalCover);
            Eos = ET.PenmanEO(SoilRadn, Met.MeanT, WindRun, Met.VP, Salb, Met.Latitude, Clock.Today.DayOfYear);
            Array.Clear(Hourly.Irrigation, 0, 24);
            Array.Clear(Hourly.Rainfall, 0, 24);
            Array.Clear(Hourly.Drainage, 0, 24);
            // enable leaching
            Array.Clear(Hourly.LeachNO3, 0, 24);
            Array.Clear(Hourly.LeachUrea, 0, 24);

            Array.Clear(Hourly.Infiltration, 0, 24);
            Array.Clear(Diffusion, 0, ProfileLayers);
            if(Plant != null)
                if(Plant.Root != null)
                    SetRootLengthDensity();

            //initialise solute transport
            // Aparrently N is already distributed to targetThickness so without Layers.MapMass is also valid
            //MappedNH4 = NH4.kgha;           
            //MappedNO3 = NO3.kgha;
            //MappedUrea = Urea.kgha;
            
            //solute at the start of the day
            MappedNH4 = Layers.MapMass(NH4.kgha, Thickness, targetThickness);
            MappedNO3 = Layers.MapMass(NO3.kgha, Thickness, targetThickness);
            MappedUrea = Layers.MapMass(Urea.kgha, Thickness, targetThickness);
            
            //initialise no3values and ureavalues for daily simulation
            for (int i = 0; i < ProfileLayers; i++)
            {
                No3Values[i] = MappedNO3[i];
                UreaValues[i] = MappedUrea[i];
            }

        }

## doInfiltration solute movement
For infiltration, only do solute transport for irrigation with solutes.

## doDrainage solute movement
For drainage, calculate solutes contained in each OutFluxCurrentLayer and move it down with the InFluxLayerBelow. This means:

soluteOutFlux = solute[l] * (OutFluxCurrentLayer/SWmm[l]) * efficiency[l]

solute[l]updated = solute[l] - soluteOutFlux

soluteInFlux = soluteOutFlux

solute[l+1]updated = solute[l+1] + soluteInFlux

In [ ]:
        /// <summary>
        /// Gravity moves mobile water out of layers each time step
        /// </summary>
        /// <param name="SPH">Steps Per Hour, the number of times this function is called in an hourly time step</param>
        /// <param name="h">h of the day for this time step</param>
        /// <param name="Subh">the current sub hourly time step</param>
        private void doDrainage(int h, int SPH, int Subh)
        {
            for (int l = 0; l < ProfileLayers; l++)
            {//Step through each layer from the top down
                double PotentialDrainage = 0;
                for (int c = PoreCompartments - 1; c >= 0; c--)
                {//Step through each pore compartment and work out how much may drain
                    PotentialDrainage += Math.Min(Pores[l][c].HydraulicConductivityOut / SPH, Pores[l][c].WaterDepth);
                }
                //Limit drainage to that of what the layer may drain and that of which the provile below will allow to drain
                double OutFluxCurrentLayer = Math.Min(PotentialDrainage, PercolationCapacityBelow[l]);
                //Catch the drainage from this layer to be the InFlux to the next Layer down the profile
                double InFluxLayerBelow = OutFluxCurrentLayer;
                //Discharge water from current layer
                for (int c = 0; c < PoreCompartments && OutFluxCurrentLayer > 0; c++)
                {//Step through each pore compartment and remove the water that drains starting with the largest pores
                    double drain = Math.Min(OutFluxCurrentLayer, Math.Min(Pores[l][c].WaterDepth, Pores[l][c].HydraulicConductivityOut / SPH));
                    Pores[l][c].WaterDepth -= drain;
                    OutFluxCurrentLayer -= drain;
                    if (ReportDetail) { DoDetailReport("Drain", l, h); }
                }
                if (Math.Abs(OutFluxCurrentLayer) > FloatingPointTolerance)
                    throw new Exception("Error in drainage calculation");

                //Calculate solutes in the drained flux out to the next layer
                double NO3OutFlux = No3Values[l] * MathUtilities.Divide(OutFluxCurrentLayer, SWmm[l], 0) * SoluteFluxEfficiency[l];
                double UreaOutFlux = UreaValues[l] * MathUtilities.Divide(OutFluxCurrentLayer, SWmm[l], 0) * SoluteFluxEfficiency[l];
                //Update solute in this layer by removing solute outflux
                No3Values[l] -= NO3OutFlux;
                UreaValues[l] -= UreaOutFlux;


                //Distribute water from this layer into the profile below and record drainage out the bottom
                //Bring the layer below up to its maximum absorption then move to the next
                for (int l1 = l + 1; l1 < ProfileLayers + 1 && InFluxLayerBelow > 0; l1++)
                {
                    //Any water not stored by this layer will flow to the layer below as saturated drainage
                    if (l1 < ProfileLayers)
                    {
                        if (ReportDetail) { DoDetailReport("Redistribute", l1, h); }
                        DistributWaterInFlux(l1, ref InFluxLayerBelow, SPH);

                        //Distribute the soluteOutFlux to the layer below
                        No3Values[l1] += NO3OutFlux;
                        UreaValues[l1] += UreaOutFlux;


                    }
                    //If it is the bottom layer, any discharge recorded as drainage from the profile
                    else
                    {
                        Hourly.Drainage[h] += InFluxLayerBelow;

                        //Remove SoluteOutFlux as Leached solute
                        Hourly.LeachNO3[h] += NO3OutFlux;
                        Hourly.LeachUrea[h] += UreaOutFlux;


                        if (SPH != 1)
                        {
                            SubHourly.Drainage[Subh] += InFluxLayerBelow;
                            //Remove SoluteOutFlux as Leached solute
                            SubHourly.LeachNO3[Subh] += NO3OutFlux;
                            SubHourly.LeachUrea[Subh] += UreaOutFlux;
                        }

                    }
                }
            }
            //Error checking for debugging.  To be removed when model complted
            UpdateProfileValues();
            CheckMassBalance("Drainage", h, SPH, Subh);
        }

## doDiffusion solute movement
For diffusion, calculate solutes contained in UpwardDiffusion and in DownwardDiffusion.

In upwardDiffusion from layer below, 

soluteInFluxUp = solute[l+1] * (UpwardDiffusion/SWmm[l+1]) * efficiency[l+1]

In downwardDiffusion to layer below,

soluteOutFluxDown = solute[l] * (DownwardDiffusion/SWmm[l]) * efficiency[l]

solute[l]updated = solute[l] + soluteInFluxUp - soluteOutFluxDown

In [ ]:
       /// <summary>
        /// Potential gradients moves water out of layers each time step
        /// </summary>
        private void doDiffusion()
        {            
            for (int l = 0; l < ProfileLayers - 1; l++)
            {//Step through each layer from the top down
                double PotentialDownwardPoiseuilleFlow = 0;
                double PotentialUpwardPoiseuilleFlow = 0;
                double DownwardDiffusion = 0;
                double UpwardDiffusion = 0;
                double DownwardDiffusionCapacity = 0;
                double UpwardDiffusionCapacity = 0;
                for (int c = 0; c < PoreCompartments; c++)
                {//Step through each pore and calculate diffusion in and out

                    PotentialDownwardPoiseuilleFlow += Pores[l][c].Diffusivity * DiffusivityMultiplier;//Diffusion out of this layer to layer below
                    UpwardDiffusionCapacity += Pores[l][c].DiffusionCapacity; //How much porosity there is in the matrix to absorb upward diffusion

                    if (l <= ProfileLayers - 1)
                    {
                        PotentialUpwardPoiseuilleFlow += Pores[l + 1][c].Diffusivity * DiffusivityMultiplier;//Diffusion into this layer from layer below
                        DownwardDiffusionCapacity += Pores[l + 1][c].DiffusionCapacity; //How much porosity there is in the matrix to absorb downward diffusion
                    }
                    else
                    {
                        PotentialUpwardPoiseuilleFlow = 0; //Need to put something here to work out capillary rise from below specified profile
                        DownwardDiffusionCapacity = 0;
                    }
                }
                UpwardDiffusion = Math.Min(PotentialUpwardPoiseuilleFlow, UpwardDiffusionCapacity);
                DownwardDiffusion = Math.Min(PotentialDownwardPoiseuilleFlow, DownwardDiffusionCapacity);
                double NetDiffusion = UpwardDiffusion - DownwardDiffusion;

                // for solute moving up from layer below
                double NO3InFluxUp = no3Values[l+1] * (UpwardDiffusion/SWmm[l+1]) * SoluteFlowEfficiency[l+1];
                double UreaInFluxUp = ureaValues[l+1] * (UpwardDiffusion/SWmm[l+1]) * SoluteFlowEfficiency[l+1];
                
                // for solute moving down from this layer
                double NO3OutFluxDown = (no3Values[l] + NO3InFluxUp) * (DownwardDiffusion/SWmm[l])*SoluteFluxEfficiency[l];
                double UreaOutFluxDown = (ureaValues[l] + UreaInFluxUp) * (DownwardDiffusion/SWmm[l])*SoluteFluxEfficiency[l];
             
                // update change in solute to this layer and to the layer below
                no3Values[l] = no3Values[l] + NO3InFluxUp - NO3OutFluxDown;
                ureaValues[l] = ureaValues[l] + UreaInFluxUp - UreaOutFluxDown;
                
                no3Values[l+1] = no3Values[l+1] - NO3InFluxUp + NO3OutFluxDown;
                ureaValues[l+1] = ureaValues[l+1] - UreaInFluxUp + UreaOutFluxDown;
                          
                Diffusion[l] += NetDiffusion;
                if (NetDiffusion > 0) //Bring water into current layer and remove from layer below
                {
                    DistributeInwardDiffusion(l, NetDiffusion);
                    if (l <= ProfileLayers - 1)
                        DistributeOutwardDiffusion(l + 1, NetDiffusion);
                }
                if (NetDiffusion < 0) //Take water out of current layer and place into layer below.
                {
                    if (l <= ProfileLayers - 1)
                        DistributeInwardDiffusion(l + 1, -NetDiffusion);
                    DistributeOutwardDiffusion(l, -NetDiffusion);
                }
            }
            UpdateProfileValues();
        }


## Set the solute 


In [ ]:

            
            # the following is updated in the doSoilMovement
            
            LeachNO3 = MathUtilities.Sum(Hourly.LeachNO3);
            LeachUrea = MathUtilities.Sum(Hourly.LeachUrea);

            // Set solute state variables.
            no3.SetKgHa(SoluteSetterType.Soil, no3Values);
            urea.SetKgHa(SoluteSetterType.Soil, ureaValues);